In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import plotly.graph_objects as go
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from plotly.subplots import make_subplots
import warnings    
warnings.filterwarnings('ignore')
import glob

start_time = datetime.now()

## Configurations

In [ ]:
# Required for all
Brand_Name = 'Brand_Name'
# dictionary 
WAMM_DICT_PATH = os.path.abspath('../../data/input_data/WMG WAMM Project Data Dictionary_v12.xlsx') 
# modeling stack
INPUT_DATA_PATH = os.path.abspath('../../data/processed_input_data/02_modelling_stack_data_seasonality.csv')
# CM_CampaignList
CM_CampaignList_Path = os.path.abspath('../../data/input_data/CMCampaignsList.xlsx') 
# Initiate excel writer
WRITER = pd.ExcelWriter('./WAMM_' + Brand_Name + '_Modeling_Documents.xlsx', engine = 'xlsxwriter') # location for the output file

# specify the model details
MODEL_DICT = {
    'sales_unit': 
                # input for modeling results
                {
                'output_folder': '../../models/manual/sales_unit/mix_media_metrics/2021-12-13_model_outputs_v2',
                'model_results_folder': '/model_results_test_2020_2021_Q1 2',     

                 # no need to change below if this is for unit model
                 'model_filter': 1,  
                 'model': 'OMNI Volume Sales Unit Model', # must follow the same naming convention
                 'subtitle': 'Sales Unit', # must follow the same naming convention
                 'outcome_var':'O_UNIT'
                },

    'new_customer': 
                # input for modeling results
                {
                'output_folder': '../../models/manual/new_customer/mix_media_metrics/2021-12-13_model_outputs_v2',
                'model_results_folder': '/model_results_test_2020_2021_Q1 2',                   

                # no need to change below if this is new customer model
                'model_filter': 2, 
                'model': 'OMNI New Customer Count Model', # must follow the same naming convention
                'subtitle': 'New Customer', # must follow the same naming convention
                'outcome_var':'O_NEW_CUSTOMER'      
                } 
}

# time period input for WMT Outcome Summary, WMC Media Spend, WMC Media Contribution  
# The start and end dates should be called from visualize ROAS during modeling, and a corresponding csv should be found in modeling folder.
# 'Period' field can be left blank.
WMC_PORTFOLIO_DICT = {
     '1': {'start_date':'2020-01-01', 'end_date':'2021-09-30', 'Period': '21 Months'},
     '2': {'start_date':'2020-01-01', 'end_date':'2020-12-31', 'Period': 'FY 2020'}, 
     '3': {'start_date':'2020-01-01', 'end_date':'2020-06-30', 'Period': 'H1 2020'}, 
     '4': {'start_date':'2021-01-01', 'end_date':'2021-06-30', 'Period': 'H1 2021'},
}

# time period input for growth analysis (due to)
# each start and end date should be called from visualize_due_calculations during modeling, and a corresponding csv should be found in modeling folder.
DUE_TO_OUTCOME_DICT = {
    'due_to_outcome_2020Q1_2021Q1': {'start_date_1':'2020-01-01', 
                                 'end_date_1':'2020-03-31', 
                                 'start_date_2':'2021-01-01', 
                                 'end_date_2':'2021-03-31'},  
    'due_to_outcome_2020_2021': {'start_date_1':'2020-01-01', 
                                 'end_date_1':'2020-09-30', 
                                 'start_date_2':'2021-01-01', 
                                 'end_date_2':'2021-09-30'}
}

# Load the stack 
data = pd.read_csv(INPUT_DATA_PATH, thousands= ",", na_values = " -   ")
data['index'] = pd.to_datetime(data['index']) 
data.set_index("index", inplace = True, drop= False) 
df = data.copy()

# Create folder to save images
if os.path.exists('./Documents_Graphs'):
    for file in os.listdir('./Documents_Graphs'):
        os.remove('./Documents_Graphs/' + file)
    print("Graphs folder cleaned.")
else:   
    os.mkdir('./Documents_Graphs')
    print("Graphs folder created.")
    

# Please clarify your custom variables that were not part of standard stack
# or name of the custom variables didn't refer to its generic alias such as AT/CT/KWB..
# Example:
# cus_var_dict = {'MY_CUSTOM_VARIABLE': ['M_SP_AB_CLK','M_SP_KWB_CLK']}


# Create customized variables based on cus_var_dict 
try: 
    for var in cus_var_dict:
        spend = var[:-3] + "SPEND"
        var_array = [0] * df.shape[0]
        spend_array = [0] * df.shape[0]
        generic_vars = cus_var_dict[var]
        for tp in generic_vars:
            var_array += df[tp]
            spend_array += df[tp[:-3] + "SPEND"]
        if var not in df.columns:
            df[var] = var_array
        if spend not in df.columns:
            df[spend] = spend_array
    has_cus_var = True
except:
    has_cus_var = False
    print("No customized variables found.")

## Helper functions

In [ ]:
# create values for the column 'Time Range' from the report, e.g. Jan'19 - Jan'21
def get_time_range(start_date_str, end_date_str):
        time_range_ls = []
        for date_str in [start_date_str, end_date_str]:
            date_time_obj = datetime.strptime(date_str, '%Y-%m-%d')
            month = date_time_obj.strftime("%b")
            year = date_time_obj.strftime("%y")
            res = "'".join([month, year])
            time_range_ls.append(res)
        time_range = " - ".join(time_range_ls)
        return time_range
    
def get_time_range_comparison(time_range_1, time_range_2):
    time_range_comparison = ' vs '.join([time_range_1, time_range_2])
    return time_range_comparison

# create values for the column 'Period' from the report, e.g. 30 Months
def get_period_value(start_date_str, end_date_str):
        start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
        end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
        num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1
        return '{} Months'.format(num_months)
    
def get_due_to_outcome_attribute(period): 
    components = period.split('_')
    if components[-3].startswith('Q'):
        idx_val_1 = ' '.join([components[-3], components[-2]])
        idx_val_2 = ' '.join([components[-3], components[-1]])
    else:
        idx_val_1 = components[-2]
        idx_val_2 = components[-1]    
    return idx_val_1,idx_val_2


wamm_mapping = pd.read_excel(WAMM_DICT_PATH, sheet_name = 'Data Dictionary')

def get_var_description_mapping(wamm_mapping):    
        return {n:d for n,d in zip(wamm_mapping['Variable Name'].values,wamm_mapping['Variable Description'].values)}

VAR_DESCRIPTION_MAPPING = get_var_description_mapping(wamm_mapping)

def get_var_description(var_name):
    if var_name in VAR_DESCRIPTION_MAPPING:
        return VAR_DESCRIPTION_MAPPING[var_name]
    elif var_name == 'Total':
        return
    else:
        return var_name

def combine_columns(row, col_A, col_B):
    if col_A in row and col_B not in row:
        return row[col_A]
    elif col_A not in row and col_B in row:
        return row[col_B]
    elif col_A in row and col_B in row:
        if pd.isna(row[col_A]):
            return row[col_B]
        else:
            return row[col_A]
    else:
        return 
    
def is_float(df, column):
    try:
        df[column] = df[column].apply(lambda x : float(x))
    except ValueError:
        return False
    return True

def is_int(val):
    try:
        num = int(val)
    except ValueError:
        return False
    return True


def rank_df(df, rank_col, coeff_rank = False):
    if coeff_rank == False:
        ranked_df = df[~df['Display Level'].isin(['Total Incremental','Base','Total'])]
        ranked_df[rank_col + '_Rank'] = ranked_df.groupby(['Model_Filter','Period Filter'])[rank_col].rank("min", ascending = False)
        cols = ['Model_Filter','Period Filter', 'Display Level',rank_col + '_Rank']
    else:
        media_var = [var for var in df['Variable Actual'] if var.startswith("M_")]    
        #print(media_var)
        ranked_df = df[df['Variable Actual'].isin(media_var)].sort_values([rank_col], ascending=[False])
        ranked_df[rank_col + '_Rank'] = ranked_df[rank_col].rank(ascending = False)
        cols = ['Variable Actual',rank_col + '_Rank'] 
    return ranked_df[cols]


def plot_waterfall(index, data, Title="", x_lab="", y_lab="",
              formatting = "{:,.1f}", green_color='#29EA38', red_color='#FB3C62', blue_color='#24CAFF',
             sorted_value = False, threshold=None, other_label='other', net_label='net', 
             rotation_value = 30, blank_color=(0,0,0,0), figsize = (10,10), save_path= None):
    '''
    Given two sequences ordered appropriately, generate a standard waterfall chart.
    Optionally modify the title, axis labels, number formatting, bar colors, 
    increment sorting, and thresholding. Thresholding groups lower magnitude changes
    into a combined group to display as a single entity on the chart.
    '''
    
    #convert data and index to np.array
    index=np.array(index)
    data=np.array(data)
    
    # wip
    #sorted by absolute value 
    if sorted_value: 
        abs_data = abs(data)
        data_order = np.argsort(abs_data)[::-1]
        data = data[data_order]
        index = index[data_order]
    
    #group contributors less than the threshold into 'other' 
    if threshold:
        
        abs_data = abs(data)
        threshold_v = abs_data.max()*threshold
        
        if threshold_v > abs_data.min():
            index = np.append(index[abs_data>=threshold_v],other_label)
            data = np.append(data[abs_data>=threshold_v],sum(data[abs_data<threshold_v]))
    
    changes = {'amount' : data}
    
    #define format formatter
    def money(x, pos):
        'The two args are the value and tick position'
        return formatting.format(x)
    formatter = FuncFormatter(money)
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.yaxis.set_major_formatter(formatter)

    #Store data and create a blank series to use for the waterfall
    trans = pd.DataFrame(data=changes,index=index)
    blank = trans.amount.cumsum().shift(1).fillna(0)
    
    trans['positive'] = trans['amount'] > 0

    #Get the net total number for the final element in the waterfall
    total = trans.sum().amount
    trans.loc[net_label]= total
    blank.loc[net_label] = total

    #The steps graphically show the levels as well as used for label placement
    step = blank.reset_index(drop=True).repeat(3).shift(-1)
    step[1::3] = np.nan

    #When plotting the last element, we want to show the full bar,
    #Set the blank to 0
    blank.loc[net_label] = 0
    
    #define bar colors for net bar
    trans.loc[trans['positive'] > 1, 'positive'] = 99
    trans.loc[trans['positive'] < 0, 'positive'] = 99
    trans.loc[(trans['positive'] > 0) & (trans['positive'] < 1), 'positive'] = 99
    
    trans['color'] = trans['positive']
    
    trans.loc[trans['positive'] == 1, 'color'] = green_color
    trans.loc[trans['positive'] == 0, 'color'] = red_color
    trans.loc[trans['positive'] == 99, 'color'] = blue_color
    
    my_colors = list(trans.color)
    
    #Plot and label
    my_plot = plt.bar(range(0,len(trans.index)), blank, width=0.5, color=blank_color)
    plt.bar(range(0,len(trans.index)), trans.amount, width=0.6,
             bottom=blank, color=my_colors)       

    #axis labels
    plt.xlabel("\n" + x_lab)
    plt.ylabel(y_lab + "\n")

    #Get the y-axis position for the labels
    y_height = trans.amount.cumsum().shift(1).fillna(0)
    
    temp = list(trans.amount)
    
    # create dynamic chart range
    for i in range(len(temp)):
        if (i > 0) & (i < (len(temp) - 1)):
            temp[i] = temp[i] + temp[i-1]
    trans['temp'] = temp
    plot_max = trans['temp'].max()
    plot_min = trans['temp'].min()
    
    #Make sure the plot doesn't accidentally focus only on the changes in the data
    if all(i >= 0 for i in temp):
        plot_min = 0
    if all(i < 0 for i in temp):
        plot_max = 0
    
    if abs(plot_max) >= abs(plot_min):
        maxmax = abs(plot_max)   
    else:
        maxmax = abs(plot_min)
        
    pos_offset = maxmax / 40
    plot_offset = maxmax / 15 ## needs to me cumulative sum dynamic

    #Start label loop
    loop = 0
    for index, row in trans.iterrows():
        # For the last item in the list, we don't want to double count
        if row['amount'] == total:
            y = y_height[loop]
        else:
            y = y_height[loop] + row['amount']
        # Determine if we want a neg or pos offset
        if row['amount'] > 0:
            y += (pos_offset*2)
            plt.annotate(formatting.format(row['amount']),(loop,y),ha="center", color = 'g', fontsize=11)
        else:
            y -= (pos_offset*4)
            plt.annotate(formatting.format(row['amount']),(loop,y),ha="center", color = 'r', fontsize=11)
        loop+=1

    #Scale up the y axis so there is room for the labels
    plt.ylim(plot_min-round(3.6*plot_offset, 7),plot_max+round(3.6*plot_offset, 7))
    
    #Rotate the labels
    plt.xticks(range(0,len(trans)), trans.index, rotation=rotation_value)
    
    #add zero line and title
    plt.axhline(0, color='black', linewidth = 0.6, linestyle="dashed")
    plt.title(Title)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(os.path.join(save_path))
        
        
header_format_features = {
            'font': 'Bogle',
              'font_color': 'white',
              'bold': False,
              'text_wrap': False,
              'valign': 'center',
              'fg_color': '#0071CE',
              'font_size': 11,
              'border': 1}

     
def add_worksheet(df, sheetname, startrow = 0, startcol = 0, float_format = "%.2f", adj_wid = True):

    df.to_excel(WRITER, sheet_name= sheetname, index= False, startrow= startrow, startcol=startcol, float_format=float_format)

    workbook  = WRITER.book   
    worksheet = WRITER.sheets[sheetname]

    #workbook.formats[0].set_font_name('Bogle')
    workbook.formats[0].set_font_size(11)
    #worksheet.hide_gridlines(2)
    #worksheet.set_zoom(90)
    header_format = workbook.add_format(header_format_features)
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(startrow, col_num + startcol, value, header_format)

    if adj_wid == True:
        for i, col in enumerate(df.columns):
            width= max(df[col].astype(str).apply(lambda x: len(x)))
            width= max([width+2, len(str(col))+2])
            worksheet.set_column(i, i, width)
        
Error_msg_ls = []

## Modeling Documentation processing 

### Sheet 1. Stack
The purpose of this block is to generate the data for sheet: Stack

In [ ]:
#cols = df.columns.tolist()[:-18]
df_stack = df.copy()
df_stack.reset_index(drop=True, inplace=True)

# Save the data with the below sheetname: 
df_stack.to_excel(WRITER, sheet_name = 'Stack', index = False)

### Sheet 2: WMT Outcome 

The purpose of this block is to generate the data for sheet: WMT Outcome

Time period is based on WMC_PORTFOLIO_DICT

In [ ]:
df_stack['date'] = pd.to_datetime(df_stack['index']) 

def agg_stack_from_cols(stack, cols, add_percent_total = True, drop_first_col = False, add_fy = False):
    
    output = pd.DataFrame(columns = ['Time Period','Variable','Total','% of Total']) 
    # add full year summary if not specified in configuration
    if add_fy == True:
        WMC_PORTFOLIO_DICT_wFY = WMC_PORTFOLIO_DICT.copy()
        dates_set_list = []
        for key, value in WMC_PORTFOLIO_DICT_wFY.items():
            dates_set_list.append({value['start_date'],value['end_date']}) 
        #print(dates_set_list)
        if {'2019-01-01', '2019-12-31'} not in dates_set_list and pd.to_datetime('2019-01-01', format = '%Y-%m-%d') >= stack.date.min() \
                                                              and pd.to_datetime('2019-12-31', format = '%Y-%m-%d') <= stack.date.max():
            WMC_PORTFOLIO_DICT_wFY['WMC_2019'] = {'start_date': '2019-01-01','end_date': '2019-12-31',\
                                                  'Period': 'FY2019'}
        if {'2020-01-01', '2020-12-31'} not in dates_set_list and pd.to_datetime('2020-01-01', format = '%Y-%m-%d') >= stack.date.min() \
                                                              and pd.to_datetime('2020-12-31', format = '%Y-%m-%d') <= stack.date.max() :
            WMC_PORTFOLIO_DICT_wFY['WMC_2020'] = {'start_date': '2020-01-01','end_date': '2020-12-31',\
                                                  'Period': 'FY2020'}
        if {'2021-01-01', '2021-12-31'} not in dates_set_list and pd.to_datetime('2021-01-01', format = '%Y-%m-%d') >= stack.date.min() \
                                                              and pd.to_datetime('2021-12-31', format = '%Y-%m-%d') <= stack.date.max():
            WMC_PORTFOLIO_DICT_wFY['WMC_2021'] = {'start_date': '2021-01-01','end_date': '2021-12-31',\
                                                  'Period': 'FY2021'}
        agg_dict = WMC_PORTFOLIO_DICT_wFY
    else:
        agg_dict = WMC_PORTFOLIO_DICT
    
    for scenario in agg_dict:
        output_sub = pd.DataFrame(columns = ['Time Period','Variable','Total'])    
        count = 0
        start_date_str = agg_dict[scenario]['start_date']
        end_date_str = agg_dict[scenario]['end_date']
        time_range = get_time_range(start_date_str, end_date_str)
        period = agg_dict[scenario]['Period']     
        stack_filtered = stack[(stack.date <= end_date_str) & (stack.date >= start_date_str)]
   
        for col in cols:
            if col not in stack_filtered.columns:
                print(col,"not found in stack!")
            else:
                var_name = col
                sum_var = stack_filtered[col].sum().round()
                output_sub.loc[count] = [period,var_name,sum_var]
                count += 1
                
        if drop_first_col == True:
            #Calculate total rather than use the first column:
            output_sub = output_sub.iloc[1:,:]
            output_sub['% of Total'] = 100*(output_sub.Total/output_sub.Total.sum())
            output_sub.loc[count] = [period,'Total',output_sub.Total.sum(),None]
        else:
            #Use the first column as the total
            output_sub['% of Total'] = 100*(output_sub.Total/output_sub.iloc[0,2])
        
        output = output.append(output_sub, ignore_index=True)
        
    if add_percent_total == False:
        output = output.iloc[:,:-1]
    return output

In [ ]:
def plot_cols(df_plot,cols, title, second_y = [False, False, False, False], divide_by = [1, 1, 1, 1]):
    num_col = len(cols)
    fig = make_subplots(specs=[[{'secondary_y': True}]])
    line1 = go.Scatter(x= df_plot.date, y= df_plot[cols[0]]/divide_by[0],
                       line=dict(color='#00BEFF'), name=cols[0], yaxis= 'y1')
    
    fig.add_trace(line1, secondary_y=second_y[0])
    
    if num_col == 1:
        fig.update_layout(title = {'text': title,'x':.5,'y':.9,'xanchor': 'center','yanchor': 'top'},legend_orientation= 'h'\
                                 ,paper_bgcolor= 'rgba(0,0,0,0)', plot_bgcolor= 'rgba(0,0,0,0)')
        fig.write_image(os.path.join('./Documents_Graphs/'+title+'.png'),
                                        width= 1000, height= 500, scale= 1)
    
    if num_col > 1:     
        line2 = go.Scatter(x= df_plot.date, y= df_plot[cols[1]]/divide_by[1],
                       line=dict(color='#FFC220'), name=cols[1], yaxis= 'y2') 
        fig.add_trace(line2, secondary_y=second_y[1])
        if num_col == 2:
            fig.update_layout(title = {'text': title,'x':.5,'y':.9,'xanchor': 'center','yanchor': 'top'},legend_orientation= 'h'\
                                 ,paper_bgcolor= 'rgba(0,0,0,0)', plot_bgcolor= 'rgba(0,0,0,0)')
            fig.write_image(os.path.join('./Documents_Graphs/'+title+'.png'),
                                        width= 1000, height= 500, scale= 1)
        elif num_col > 2:
            line3 = go.Scatter(x= df_plot.date, y= df_plot[cols[2]]/divide_by[2],
                           line=dict(color='#70AD47'), name=cols[2], yaxis= 'y1')
            fig.add_trace(line3, secondary_y=second_y[2])
            if num_col == 3:
                fig.update_layout(title = {'text': title,'x':.5,'y':.9,'xanchor': 'center','yanchor': 'top'},legend_orientation= 'h'\
                                 ,paper_bgcolor= 'rgba(0,0,0,0)', plot_bgcolor= 'rgba(0,0,0,0)')
                fig.write_image(os.path.join('./Documents_Graphs/'+title+'.png'),
                                            width= 1000, height= 500, scale= 1)
            elif num_col > 3:     
                line4 = go.Scatter(x= df_plot.date, y= df_plot[cols[3]]/divide_by[3],
                               line=dict(color='#f79a36'), name=cols[3], yaxis= 'y2') 
                fig.add_trace(line4, secondary_y=second_y[3])
                fig.update_layout(title = {'text': title,'x':.5,'y':.9,'xanchor': 'center','yanchor': 'top'},legend_orientation= 'h'\
                                 ,paper_bgcolor= 'rgba(0,0,0,0)', plot_bgcolor= 'rgba(0,0,0,0)')
                fig.write_image(os.path.join('./Documents_Graphs/'+title+'.png'),
                                            width= 1000, height= 500, scale= 1)


In [ ]:
#Aggregate outcomes per time period specified in WMC_PORTFOLIO_DICT

Unit_Outcome = agg_stack_from_cols(df_stack,['O_UNIT','O_UNIT_STORE','O_UNIT_DOTCOM','O_UNIT_OG'], add_fy = True)
add_worksheet(Unit_Outcome, sheetname = 'WMT Outcome', startrow = 1)
worksheet = WRITER.sheets['WMT Outcome']
worksheet.write_string('A1', 'Units By Conversion Channel:')

New_Cus_Outcome = agg_stack_from_cols(df_stack,['O_NEW_CUSTOMER','O_UNIT'], add_percent_total = False, add_fy = True)
add_worksheet(New_Cus_Outcome, sheetname = 'WMT Outcome', startrow = 26)#Unit_Outcome.shape[1]+7)
worksheet.write_string('A26', 'Units VS New Customers:')

Sales_Outcome = agg_stack_from_cols(df_stack,['O_SALE','O_SALE_STORE','O_SALE_DOTCOM','O_SALE_OG'], add_fy = True)
add_worksheet(Sales_Outcome, sheetname = 'WMT Outcome', startrow = 51)
worksheet.write_string('A51', 'Sales By Conversion Channel:')

#Plot Time Series
plot_cols(df_stack,['O_UNIT','O_UNIT_STORE','O_UNIT_DOTCOM','O_UNIT_OG'],'Units By Conversion Channel',[False,False,True,True])
plot_cols(df_stack,['O_NEW_CUSTOMER','O_UNIT'],'Units VS New Customers',[False,True])
plot_cols(df_stack,['O_SALE','O_SALE_STORE','O_SALE_DOTCOM','O_SALE_OG'],'Sales By Conversion Channel',[False,False,True,True])
        
#Add plots to excel sheets
worksheet.insert_image('G1','./Documents_Graphs/Units By Conversion Channel.png')
worksheet.insert_image('G25','./Documents_Graphs/Units VS New Customers.png')
worksheet.insert_image('G50','./Documents_Graphs/Sales By Conversion Channel.png')

#Add actual outcomes if not modeling on sales_unit or new customer

actual_outcomes = [] 
for model in MODEL_DICT:
    outcome = MODEL_DICT[model]['outcome_var']
    if outcome not in ['O_UNIT','O_NEW_CUSTOMER']:
        actual_outcomes.append(outcome)
        
if len(actual_outcomes)>0:
    Actual_Outcome = agg_stack_from_cols(df_stack,actual_outcomes, add_fy = True)
    add_worksheet(Actual_Outcome, sheetname = 'WMT Outcome', startrow = 72)
    worksheet.write_string('A72', 'Actual outcomes:')
    plot_cols(df_stack,actual_outcomes,'Actual Outcomes')
    worksheet.insert_image('G72','./Documents_Graphs/Actual Outcomes.png')


### Sheet 3: WMC Media Spend

In [ ]:
wmc_spend = agg_stack_from_cols(df_stack,['M_TOTAL_MEDIA_SPEND','M_ON_DIS_AT_SPEND','M_ON_DIS_CT_SPEND','M_ON_DIS_KW_SPEND',\
                                          'M_ON_DIS_HPLO_SPEND','M_ON_DIS_ROS_SPEND','M_OFF_DIS_WN_SPEND',\
                                      'M_SP_AB_SPEND','M_SP_KWB_SPEND','M_SBA_SPEND','M_ON_DIS_SEA_SPEND',\
                                        'M_INSTORE_TV_WALL_SPEND','M_OFF_DIS_SMEDIA_SPEND','M_INSTORE_SCO_SPEND'], drop_first_col = True, add_fy = True)


wmc_spend['WAMM Tactic'] = wmc_spend['Variable'].apply(get_var_description)
wmc_spend.rename(columns = {'Total': 'Total Spend'}, inplace = True) 

In [ ]:
#wmc_spend.to_excel(WRITER, sheet_name = 'WMC Media Spend', index = False, float_format="%.2f")
add_worksheet(wmc_spend[['Time Period','Variable', 'WAMM Tactic','Total Spend', '% of Total']],sheetname = 'WMC Media Spend')

### Sheet 4: Coefficient Summary

In [ ]:
# get the act_pred_comparison dataframe for one model
def get_model_summary_dataframe(model):
    # read the model_output 
    base_path = MODEL_DICT[model]['output_folder']+MODEL_DICT[model]['model_results_folder']
    file_path = os.path.join(base_path, 'model_outputs.xlsx')
    try:
        coeff_df = pd.read_excel(file_path, sheet_name = 'Coefficient Summary')  
        coeff_df['Model'] = MODEL_DICT[model]['model']
        coeff_df_rank = rank_df(coeff_df,'Coefficient', coeff_rank = True)
        coeff_df = pd.merge(coeff_df, coeff_df_rank, how='left', on=['Variable Actual'])

        coeff_df.rename(columns = {'Variables': 'Tactic', 
                                         'Variable Actual': 'Variable'}, inplace = True) 
        coeff_df['model_key'] = model
        return coeff_df
    except:
        msg = 'Coefficient ' + file_path + ' not found.'
        Error_msg_ls.append(msg)
        print(msg)

In [ ]:
# combine the dataframes from all models
def combine_model_summary_dataframes():
    model_summary_ls = []
    for model in MODEL_DICT:
        modelling_coeff = get_model_summary_dataframe(model)
        model_summary_ls.append(modelling_coeff)
    
    coeff_df = pd.concat(model_summary_ls)
   
    return coeff_df

In [ ]:
# combine the dataframes from all models
combined_modelling_summary_df = combine_model_summary_dataframes()

# Save the data with the below sheetname: 
cols = ['Model','Tactic','Variable', 'Coefficient', 'p-value', 'Significance', 'Elasticity',
   'Standardized Coefficient', 'Relative Importance', 'VIF','Coefficient_Rank'] 
add_worksheet(combined_modelling_summary_df[cols],sheetname = 'Coefficient Summary',float_format = "%.5f")

### Sheet 5: Model Fit

In [ ]:
def get_modelling_metrics_dataframe(model):
    # read the model_output
    base_path = MODEL_DICT[model]['output_folder']+MODEL_DICT[model]['model_results_folder']
    file_path = os.path.join(base_path, 'model_outputs.xlsx')
    modelling_metrics = pd.read_excel(file_path, sheet_name = 'Model Stats')
    modelling_metrics['Model Filter'] = MODEL_DICT[model]['model_filter']
    modelling_metrics['Model'] = MODEL_DICT[model]['model']
    modelling_metrics = modelling_metrics.reset_index(drop = True)
    modelling_metrics['Model_Dict_Name'] = model
    # reorder the columns
    cols = ['Model_Dict_Name','Model Filter', 'Model', 'Metric', 'Model Fit', 'Validation Fit']
    return modelling_metrics[cols]

In [ ]:
def combine_modelling_metrics_dataframes():
    modelling_metrics_ls = []
    for model in MODEL_DICT:
        modelling_metrics = get_modelling_metrics_dataframe(model)
        modelling_metrics_ls.append(modelling_metrics)
        
    modelling_metrics_df = pd.concat(modelling_metrics_ls)
    return modelling_metrics_df

In [ ]:
# combine the dataframes from all models and save to excel
#WRITER = pd.ExcelWriter('./WAMM_' + Brand_Name + '_Modeling_Documents.xlsx', engine = 'xlsxwriter')
combined_modelling_metrics_df = combine_modelling_metrics_dataframes()
count = 0
for model in list(combined_modelling_metrics_df.Model_Dict_Name.unique()):
    
    base_path = MODEL_DICT[model]['output_folder']+MODEL_DICT[model]['model_results_folder']
    graph_path = base_path+'/graphs/Actual vs Predicted.png'
    # Read files from model output folder
    file_path = os.path.join(base_path, 'model_outputs.xlsx')
    modelling_df = pd.read_excel(file_path, sheet_name = 'Act vs Pred Modelling')
    modelling_df['date'] = pd.to_datetime(modelling_df['index']) 
    validation_df = pd.read_excel(file_path, sheet_name = 'Act vs Pred Validation')
    validation_df['date'] = pd.to_datetime(validation_df['index']) 
    model_fit_df = combined_modelling_metrics_df[combined_modelling_metrics_df.Model_Dict_Name == model].iloc[:,2:]

    # Add to worksheet
    add_worksheet(model_fit_df,sheetname = 'Model Fit',\
                 startrow= count *30)
    worksheet = WRITER.sheets['Model Fit']
    windwow_df = pd.DataFrame({'Training Start': modelling_df.date.min().strftime('%Y-%m-%d'),
 'Training End': modelling_df.date.max().strftime('%Y-%m-%d'),
 'Validation Start': validation_df.date.min().strftime('%Y-%m-%d'),
 'Validation End': validation_df.date.max().strftime('%Y-%m-%d')}, index=['i',])
    add_worksheet(windwow_df,sheetname = 'Model Fit',\
                 startrow= count *30, startcol = 4)
    worksheet.write_string(count *30,9, model+' model:')
    worksheet.insert_image(count *30,10,graph_path,{'x_scale': .2, 'y_scale': .2})
    count += 1
#WRITER.save()

### Sheet 6:  WMC Media Contribution
The purpose of this block is to generate the data for sheet: WMC Media Contribution

Time period is based on WMC_PORTFOLIO_DICT.

In [ ]:
# get per period/scenario WMC dataframe
def get_per_scenario_WMC_dataframe(model, scenario, period_filter):
    
    base_path = MODEL_DICT[model]['output_folder']
    start_date_str = WMC_PORTFOLIO_DICT[scenario]['start_date']
    end_date_str = WMC_PORTFOLIO_DICT[scenario]['end_date']
    sub_path = 'WMC_' + str(start_date_str) + '_'+ str(end_date_str) + '.csv'
    file_path = os.path.join(base_path, sub_path)  
    
    try:
        WMC_df = pd.read_csv(file_path)
        WMC_df['Model_Filter'] = MODEL_DICT[model]['model_filter']
        WMC_df['Model'] = MODEL_DICT[model]['model']
        WMC_df['Subtitle'] = MODEL_DICT[model]['subtitle']
        WMC_df['Period Filter'] = period_filter
        WMC_df['Time Range'] = get_time_range(start_date_str, end_date_str)
        WMC_df['Period'] = WMC_PORTFOLIO_DICT[scenario]['Period']
        #print(WMC_df)
        return WMC_df
    except:
        msg = 'Warning! {model} model - {scenario}: {sub_path} does not exist! Please double check the start and end dates have been used in modeling!\n'.format(model = model, scenario=scenario, sub_path = sub_path)
        Error_msg_ls.append(msg)
        print(msg)
        print(file_path)
        return

In [ ]:
# update get per period/scenario WMC media contribution dataframe
def update_per_scenario_WMC_media_contribution_dataframe(WMC_media_contribution_df):
    if WMC_media_contribution_df is None:
        return 
    WMC_media_contribution_df['Value RoAS ($)'] = WMC_media_contribution_df.apply (lambda row: combine_columns(row, 'Value NCoAS ($)', 'Value RoAS ($)'), axis=1)
    WMC_media_contribution_df['Volume RoAS'] = WMC_media_contribution_df.apply (lambda row: combine_columns(row, 'NCoAS', 'Volume RoAS'), axis=1)
    #WMC_media_contribution_df = WMC_media_contribution_df[~WMC_media_contribution_df['Display Level'].isin(['Total Incremental', 'Base', 'Total'])]
        
    # convert the string columns into float
    str_cols = ['Total Contribution', 'Total Sales ($)', 'Total Spend ($)', 'Raw Media']
    #check if the column could be converted to float, if yes, convert to float
    for str_col in str_cols:   
        WMC_media_contribution_df[str_col] = WMC_media_contribution_df[str_col].apply(lambda x : x.replace(',', ''))
        if is_float(WMC_media_contribution_df, str_col):
            WMC_media_contribution_df[str_col] = WMC_media_contribution_df[str_col].apply(lambda x : float(x))
    WMC_media_contribution_df.replace([np.inf, -np.inf], np.nan, inplace=True)    
    # reorder the DataFrame based on the columns
    cols = ['Model', 'Model_Filter','Period','Period Filter', 'Time Range', 'Display Level',
            'Contribution Share (%)', 'Total Contribution', 'Total Sales ($)', 'Total Spend ($)', 
           'Spend Share (%)', 'Raw Media', 'Raw Media Share (%)', 'Volume RoAS', 'Value RoAS ($)']
    return WMC_media_contribution_df[cols]

In [ ]:
# get all_scenario_WMC_media_contribution dataframes from one model
def get_per_model_WMC_media_contribution_dataframe(model):
    per_model_WMC_media_contribution_ls = []
    period_filter = 1
    for scenario in WMC_PORTFOLIO_DICT:
        WMC_media_contribution_df = get_per_scenario_WMC_dataframe(model, scenario, period_filter)
        if WMC_media_contribution_df is not None:
            updated_WMC_media_contribution_df = update_per_scenario_WMC_media_contribution_dataframe(WMC_media_contribution_df)
            per_model_WMC_media_contribution_ls.append(updated_WMC_media_contribution_df)
            period_filter += 1
    if per_model_WMC_media_contribution_ls:
        per_model_WMC_media_contribution_df = pd.concat(per_model_WMC_media_contribution_ls)
        return per_model_WMC_media_contribution_df

In [ ]:
# combine dataframes from all models
def combine_WMC_media_contribution_dataframes():
    WMC_media_contribution_df_all = pd.DataFrame() 
    
    for model in MODEL_DICT:
        WMC_media_contribution_ls = []
        #print(model)
        per_model_WMC_media_contribution = get_per_model_WMC_media_contribution_dataframe(model)
        WMC_media_contribution_ls.append(per_model_WMC_media_contribution)
        #print(model)
        if  WMC_media_contribution_ls:
            WMC_media_contribution_df = pd.concat(WMC_media_contribution_ls)
        #print(WMC_media_contribution_df)
        contribution_rank = rank_df(WMC_media_contribution_df,'Total Contribution')
        roas_rank = rank_df(WMC_media_contribution_df,'Value RoAS ($)')
        WMC_media_contribution_df = pd.merge(WMC_media_contribution_df, contribution_rank, how='left', on=['Model_Filter','Period Filter', 'Display Level'])
        WMC_media_contribution_df = pd.merge(WMC_media_contribution_df, roas_rank, how='left', on=['Model_Filter','Period Filter', 'Display Level'])
        
        WMC_media_contribution_df_all = WMC_media_contribution_df_all.append(WMC_media_contribution_df)
    
    WMC_media_contribution_df_all.rename(columns = {'Total Contribution_Rank': 'Contribution Rank', 
                                         'Value RoAS ($)_Rank': 'RoAS Rank'}, inplace = True)

    return WMC_media_contribution_df_all

In [ ]:
# update the WMC media contribution dataset
combined_WMC_media_contribution_df = combine_WMC_media_contribution_dataframes()

#Save the data with the below sheetname: 
  
for model_filter in combined_WMC_media_contribution_df.Model_Filter:

    model1_df = combined_WMC_media_contribution_df[combined_WMC_media_contribution_df.Model_Filter == model_filter]
    add_worksheet(model1_df,sheetname = 'WMC Media Contribution',startrow=(int(model_filter)-1)*(model1_df.shape[0]+20), startcol=0,float_format = "%.3f")


### Sheet 7-:  WMC Media Summary by Tactic

The purpose of this block is to generate sheets that summarize models by tactic.

All metrics are based on training + validation window.

In [ ]:
# Pulls contribution, coefficients, aring analysis, and line item summary based on input model and TP
def tactic_summary(df,model, TP_original, corr_outcome):
    indices = [i for i, x in enumerate(TP_original) if x == "_"]
    TP = TP_original[:indices[-5]]
    
    df_copy = df.copy()
    # Filter for modeling window 
    df = df[(df.date <= windwow_df['Validation End'][0])&(df.date >= windwow_df['Training Start'][0])]

    tactic = get_var_description(TP)
    spend_var = TP[:-3]+'SPEND'
    
    if TP not in list(wamm_mapping['Variable Name']):
        msg = TP+" not found in Data Dictionary. Please populate it in Data Dictionary and rerun the notebook."
        Error_msg_ls.append(msg)
        print(msg)
    
    elif TP not in df.columns:
        msg = TP+" not found in stack. Please add it through cus_var_dict in configuration and rerun the notebook."
        Error_msg_ls.append(msg)
        print(msg)
        
    elif spend_var not in df.columns:
        msg = spend_var+" not found in stack. Please add it through cus_var_dict in configuration and rerun the notebook."
        Error_msg_ls.append(msg)
        print(msg)
    
    else:
        Model_Filter = MODEL_DICT[model]['model_filter']
        # Correlation with Outcome
        correlation = np.corrcoef(df[TP],df[corr_outcome])[0,1]

        # Contribution
        tp_contribution = combined_WMC_media_contribution_df[(combined_WMC_media_contribution_df.Model_Filter == Model_Filter)&\
                                            (combined_WMC_media_contribution_df['Period Filter'] == 1)&\
                                            (combined_WMC_media_contribution_df['Display Level'] == tactic)]

        tp_contribution['Corr with Outcome'] = correlation
        tp_contribution = tp_contribution[['Model','Display Level','Corr with Outcome','Total Contribution','Total Sales ($)','Contribution Share (%)','Raw Media','Total Spend ($)',\
                                           'Value RoAS ($)','Contribution Rank','RoAS Rank']]
        model_name = tp_contribution.Model.unique()[0]

        # Effectiveness
        tp_effectiveness = combined_modelling_summary_df[(combined_modelling_summary_df.Model == model_name)&\
                                            (combined_modelling_summary_df['Variable'].str.strip() == TP_original.strip())]
        tp_effectiveness = tp_effectiveness[['Variable','Coefficient','Standardized Coefficient','Significance','Coefficient_Rank']]
        trans_tp = tp_effectiveness.iloc[0,0]

        # Airing Anlaysis
        base_path = MODEL_DICT[model]['output_folder']+MODEL_DICT[model]['model_results_folder']
        file_path = os.path.join(base_path, 'variables_transformation_summary.xlsx')
        aring_df = pd.read_excel(file_path, sheet_name = 'Airing Analysis')   
        aring_df = aring_df[aring_df.Variable == TP_original]
        aring_df = aring_df.iloc[:,1:]

        # Time Series Plot with Outcome - Original Variable and Spend
        if TP.endswith("_CLK"):
            plot_cols(df,[corr_outcome,TP,spend_var], title = TP+","+corr_outcome,second_y = [True,False,False])
        else:
            plot_cols(df,[corr_outcome,TP,spend_var], title = TP+","+corr_outcome,second_y = [True,False,False],divide_by = [1,1,1])
            
        #Time Series Plot with Outcome - Transformed variable
        if trans_tp in df.columns:
            # plot final transformed variable from stack if exists
            plot_cols(df,[corr_outcome,trans_tp], title = trans_tp+","+corr_outcome,second_y = [True,False])
            return tp_contribution, tp_effectiveness, aring_df, trans_tp
        else:
            try: # plot transformed variable from output folder
                training_df = df[df.date <= windwow_df['Training End'][0]]
                s_cruve_transformd_df = pd.read_excel(file_path, sheet_name = 'Transformations')
                s_cruve_transformd_df[corr_outcome] = list(training_df[corr_outcome])
                s_cruve_transformd_df['date']  = list(training_df['date'])
                plot_cols(s_cruve_transformd_df,[corr_outcome,tactic], title = TP+"(transformed),"+corr_outcome,second_y = [True,False])

                return tp_contribution, tp_effectiveness, aring_df, trans_tp
            except:
                msg = 'Failed to plot transformed variable:'+TP+' for model: '+model
                Error_msg_ls.append(msg)
                print(msg)
                return tp_contribution, tp_effectiveness, aring_df, trans_tp

In [ ]:
# Aggregate line item metircs from CMCampaignsList
def agg_line_item(generic_tactic,path):

    cp_list = pd.read_excel(path, sheet_name = 'CampaignList')   

    # Split generic tactic keywords and find the actual name in CMCampaignslist
    generic_tactic_split = generic_tactic.upper().split("_")
    for var in list((cp_list['WAMM Tactic'].str.strip().unique())):
        var_upper = var.upper()
        kw_count = 0
        for item in generic_tactic_split:
            if item in var_upper:
                kw_count += 1
        if (kw_count == len(generic_tactic_split))&((("ROLL" in var_upper)&("ROLL" in generic_tactic.upper()))|(("ROLL" not in var_upper)&("ROLL" not in generic_tactic.upper()))):
            actual_tactic = var
            
    # Filter CMCampaignsList based on tactic
    tp_cp_list = cp_list[(cp_list['WAMM Tactic'].str.strip() == actual_tactic.strip())&(cp_list.Related_Brand == 'Yes')]
    
    stack_spend = []
    if 'Spend_billable' in tp_cp_list.columns:
        spend_billed = list(tp_cp_list['Spend_billable'])
    else:
        spend_billed = list(tp_cp_list['Spend_billed'])
    spend_delivered = list(tp_cp_list['Spend_delivered'])

    i = 0
    for sp in tp_cp_list.Related_Spend:
        if (sp == 'Spend_billable') or (sp == 'Spend_billed'):
            stack_spend.append(spend_billed[i])
        else :
            stack_spend.append(spend_delivered[i])
        i += 1

    tp_cp_list['Stacked_Spend'] = stack_spend
    output = tp_cp_list.groupby(['line_item_start_date','line_item_end_date','campaign_name','line_item_name']).\
    agg({'Stacked_Spend':'sum'}).reset_index()
    
    output['Tactic'] = actual_tactic
    
    return output

In [ ]:
# This function aims to find the WAMM tactic name in CMCampaignsList of the given TP
# It can process customized TP too. eg: M_ON_DIS_CT_AT_IMP 
def TP_to_generic(TP):
    metric = TP[-3:]
    generic_tactics = [] 
    generic_variables = []
    if '_CT_' in TP:
        generic_tactics.append('Display_contextual_targeting')
        generic_variables.append("M_ON_DIS_CT_"+metric)
    if '_AT_' in TP:
        generic_tactics.append('Display_audience_targeting')
        generic_variables.append("M_ON_DIS_AT_"+metric)
    if '_ROS_' in TP:
        generic_tactics.append('Display_run_of_site')
        generic_variables.append("M_ON_DIS_ROS_"+metric)
    if '_DIS_KW_' in TP:
        generic_tactics.append('Display_keyword')
        generic_variables.append("M_ON_DIS_KW_"+metric)
    if '_HPLO_' in TP:
        generic_tactics.append('Display_homepage')
        generic_variables.append("M_ON_DIS_HPLO_"+metric)
    if '_WN_DISPLAY_' in TP:
        generic_tactics.append('Display_web_apps')
        generic_variables.append("M_OFF_DIS_WN_DISPLAY_"+metric)
    if '_WN_PREROLL_' in TP:
        generic_tactics.append('Display_web_apps_pre_roll')
        generic_variables.append("M_OFF_DIS_WN_PREROLL_"+metric)
    if '_SBA_' in TP:
        generic_tactics.append('Search_Brand_Amplifier')
        generic_variables.append("M_SBA_"+metric)
    if '_INSTORE_TV_' in TP:
        generic_tactics.append('TV_Wall')
        generic_variables.append("M_INSTORE_TV_WALL_"+metric)
    if '_INSTORE_SCO_' in TP:
        generic_tactics.append('Self_Check_out')
        generic_variables.append("M_INSTORE_SCO_"+metric)
    if '_FB_' in TP:
        generic_tactics.append('Display_Facebook')  
        generic_variables.append("M_ON_DIS_FB_"+metric)
    if '_PIN_' in TP:
        generic_tactics.append('Display_Pinterest') 
        generic_variables.append("M_OFF_DIS_PIN_"+metric)
    if '_SEA_' in TP:
        generic_tactics.append('Display_seasonal') 
        generic_variables.append("M_ON_DIS_SEA_"+metric)
    if '_AB_' in TP:
        generic_tactics.append('Sponsored_Products_automatic')
        generic_variables.append("M_SP_AB_"+metric)
    if '_KWB_' in TP:
        generic_tactics.append('Sponsored_Products_manual') 
        generic_variables.append("M_SP_KWB_"+metric)
    # Known aggregated TP:
    if TP == 'M_SP_CLK' or TP == 'M_SP_IMP':
        generic_tactics = generic_tactics + ['Sponsored_Products_automatic','Sponsored_Products_manual']
        generic_variables = generic_variables + ['M_SP_AB_'+metric,'M_SP_KWB_'+metric]
    if TP =='M_OFF_DIS_SMEDIA_'+metric:
        generic_tactics = generic_tactics + ['Display_Facebook','Display_Pinterest']
        generic_variables = generic_variables + ['M_ON_DIS_FB_'+metric,'M_OFF_DIS_PIN_'+metric]
    if TP == 'M_OFF_DIS_WN_'+metric:
        generic_tactics = generic_tactics + ['Display_web_apps','Display_web_apps_pre_roll']
        generic_variables = generic_variables + ['M_OFF_DIS_WN_DISPLAY_'+metric,'M_OFF_DIS_WN_PREROLL_'+metric]
    if TP == 'M_SEARCH_MAN_'+metric:
        generic_tactics = generic_tactics + ['Search_Brand_Amplifier','Sponsored_Products_manual']
        generic_variables = generic_variables + ['M_SBA_'+metric,'M_SP_KWB_'+metric]
    if TP == 'M_SEARCH_AUTO_'+metric:
        generic_tactics = generic_tactics + ['Sponsored_Products_automatic']
        generic_variables = generic_variables + ['M_SP_AB_'+metric]

    return generic_tactics, generic_variables

In [ ]:
# Print tactic summaries to excel
from collections import defaultdict
sheet_item_count = defaultdict(int)

def print_model_tactic_summary(model, TP_original, CM_List_path = None):
    
    indices = [i for i, x in enumerate(TP_original) if x == "_"]
    TP = TP_original[:indices[-5]]
    tactic = get_var_description(TP)
    outcome_var = MODEL_DICT[model]['outcome_var']
    sheetname = tactic.replace("Onsite Display",'').title()

    if sheetname in WRITER.book.sheetnames:
        count = sheet_item_count[sheetname]
    else:
        count = 0
    
    generic_tactics = TP_to_generic(TP)[0]
    generic_vars = TP_to_generic(TP)[1] 
    
    # For variables specified in cus_var_dict, use the specification to get generic variables and tactics
    if has_cus_var == True:
        if TP in cus_var_dict:
            generic_vars = cus_var_dict[TP]
            generic_tactics = []
            for v in generic_vars:
                generic_tactics += TP_to_generic(v)[0]
    # Check whether the generic variables of a given TP is found
    if len(generic_vars) == 0:
        msg = "Warning: Generic tactic of variable " + TP +" not found. Line item information will be missing from the output."
        Error_msg_ls.append(msg)
        print(msg)

    try:
        tp_contribution, tp_coeff,tp_aring, trans_tp = tactic_summary(df_stack,model, TP_original, outcome_var)
        tp_coeff['Generic Variables'] = ",".join(generic_vars)
        tp_contribution.rename(columns = {'Display Level': 'Tactic'}, inplace = True)

        # Write tactic summary tables to excel 
        add_worksheet(tp_contribution, sheetname, startrow = 1+count*40, startcol = 0, float_format = "%.2f")
        add_worksheet(tp_coeff, sheetname, startrow = 5+count*40, startcol = 0, float_format = "%.5f")
        add_worksheet(tp_aring, sheetname, startrow = 10+count*40, startcol = 0, float_format = "%.3f")      
        worksheet = WRITER.sheets[sheetname]
        worksheet.write_string(count *40,0, 'Contribution Details:')
        worksheet.write_string(count *40+4,0,'Coefficient Effectiveness:')
        worksheet.write_string(count *40+9,0,'Airing Analysis:')

        # Add TP,Spend and Outcome 
        if TP.endswith("CLK"):
            title = TP+","+outcome_var
        else:
            title = TP+","+outcome_var
        worksheet.insert_image(count*40+12,0,'./Documents_Graphs/'+title+'.png',{'x_scale': .9, 'y_scale': .9})
        
        # Add transformed TP and outcome 
        worksheet.insert_image(count*40+12,6,'./Documents_Graphs/'+trans_tp+","+outcome_var+'.png',{'x_scale': .9, 'y_scale': .9})
        worksheet.insert_image(count*40+12,6,'./Documents_Graphs/'+TP+"(transformed),"+outcome_var+'.png',{'x_scale': .9, 'y_scale': .9})

        # Add S curve  
        base_path = MODEL_DICT[model]['output_folder']+MODEL_DICT[model]['model_results_folder']
        file_path = base_path+'/graphs/Response curve - '+tactic+'.png'
        worksheet.insert_image(count*40+12,12,file_path,{'x_scale': .15, 'y_scale': .15})
        sheet_item_count[sheetname] += 1
    except:
        msg = 'Preparing summary of '+TP+' for '+ model+' model failed.'
        Error_msg_ls.append(msg)
        
    # Append line_item info from CMCampaignsList
    if CM_List_path != None:
        tactic_count = 0
        for i in range(len(generic_tactics)):
            try:
                line_item_summary = agg_line_item(generic_tactics[i],CM_List_path)
                if sum(line_item_summary.Stacked_Spend) > 0:
                    add_worksheet(line_item_summary, sheetname, startrow = 0, startcol = 25 + 7* tactic_count, float_format = "%.2f", adj_wid = False)
                    tactic_count += 1
            except:
                msg = 'Warning! Cannot find '+get_var_description(generic_vars[i])+' from CM_CampaignsList.'
                Error_msg_ls.append(msg)
                print(msg)

In [ ]:
# Get WMC media variables from coefficient summary w/o adstock parameters
def get_media_var_list_per_model(combined_modelling_summary_df):
    
    media_vars = [var for var in list(combined_modelling_summary_df.Variable.unique()) if (var.startswith("M_")&\
                                                                                           ("NATIONAL" not in var.upper())&\
                                                                                           ("WMT" not in var.upper()))]
    sub_df = combined_modelling_summary_df.loc[:,['model_key','Variable']]
    sub_df = sub_df[sub_df.Variable.isin(media_vars)]
    
    #media_var_ls_sort = sorted(set(clean_media_var_ls), key=clean_media_var_ls.index)
    return sub_df  

In [ ]:
#WRITER = pd.ExcelWriter('./WAMM_' + 'test' + '_Modeling_Documents.xlsx', engine = 'xlsxwriter') # location for the output file
for row in range(get_media_var_list_per_model(combined_modelling_summary_df).shape[0]):
    
    model_key = get_media_var_list_per_model(combined_modelling_summary_df).iloc[row,0]
    TP = get_media_var_list_per_model(combined_modelling_summary_df).iloc[row,1]
    
    print("Preparing",TP,"for model:", model_key)
    print_model_tactic_summary(model_key, TP, CM_List_path = CM_CampaignList_Path)
        
#WRITER.save()

### Sheet: Base VS Incremental

In [ ]:
def model_base_incremental(model, model_count, start_col):
    # Get base incremental df from model outputs
    base_path = MODEL_DICT[model]['output_folder']+MODEL_DICT[model]['model_results_folder']
    file_path = os.path.join(base_path, 'model_outputs.xlsx')
    base_incre_df = pd.read_excel(file_path, sheet_name = 'Base - Incrmental Contribution')  
    df_col = base_incre_df.shape[1]+2
    sheetname = 'Base vs Incremental'
    add_worksheet(base_incre_df, sheetname = sheetname, startrow = 1, startcol = start_col)

  # Add Base incremental chart  
    worksheet = WRITER.sheets[sheetname]
    base_path = MODEL_DICT[model]['output_folder']+MODEL_DICT[model]['model_results_folder']
    file_path = base_path+'/graphs/Base vs Incremental Shade Plot.png'
    worksheet.write_string(0,start_col,MODEL_DICT[model]['model'])
    worksheet.insert_image(10,start_col,file_path,{'x_scale': .2, 'y_scale': .2})
    
    return df_col
    

In [ ]:
#WRITER = pd.ExcelWriter('./WAMM_' + Brand_Name + '_Modeling_Documents.xlsx', engine = 'xlsxwriter') # location for the output file
# Add base vs incremental for all models:
model_count = 0
start_col = 0
for model in MODEL_DICT:
    start_col = model_base_incremental(model, model_count,start_col)
    model_count += 1
    
#WRITER.save()

### Sheet: Growth Analysis

In [ ]:
# get per period/scenario due_to_outcome dataframe and plot due to waterfall
def get_per_scenario_due_to_outcome_dataframe(model, scenario):
    
    base_path = MODEL_DICT[model]['output_folder']
    start_date_str_1 = DUE_TO_OUTCOME_DICT[scenario]['start_date_1']
    end_date_str_1 = DUE_TO_OUTCOME_DICT[scenario]['end_date_1']
    start_date_str_2 = DUE_TO_OUTCOME_DICT[scenario]['start_date_2']
    end_date_str_2 = DUE_TO_OUTCOME_DICT[scenario]['end_date_2']
    sub_path = 'WMC_Due_to_' + str(start_date_str_1) + '_' + str(end_date_str_1) + '_' + str(start_date_str_2) + '_'+ str(end_date_str_2)+ '.csv'
    path = os.path.join(base_path, sub_path)
    #Read due to csv from model output folder
    try:
        model_output_df = pd.read_csv(path)
        model_output_df.columns = ['Drivers','Contribution Period 1','Contribution Period 2','Due To','Due To (%)','Spend Period 1','Spend Period 2']
        df = model_output_df.copy()
    except:
        msg = 'Warning! {model} model - {scenario}: {sub_path} does not exist! Please double check the start and end dates!\n'.format(model = model, scenario=scenario, sub_path = sub_path)
        Error_msg_ls.append(msg)
        print(msg)
        
    #Prepare df for waterfall chart
    try:    
        due_tos = df['Due To'].tolist()
        #numbers are saved as string. convet to numetic for aggregation.
        due_tos = [num.replace(',','') for num in due_tos]
        df['Due To'] = due_tos
        df['Due To'] = df['Due To'].astype('float').tolist()
        period1_outcome = float(df.loc[(df.iloc[:,0] == 'Outcome Variable'),'Contribution Period 1'][0].replace(',',''))
        period2_outcome =float(df.loc[(df.iloc[:,0] == 'Outcome Variable'),'Contribution Period 2'][0].replace(',',''))
        #Calculate non media drivers due tos including unexpained part(caused by difference of prediction and actual)
        df = df.iloc[:,[0,3]]
        df2=pd.DataFrame([['Unexplained',period2_outcome-period1_outcome-sum(df.loc[(df.iloc[:,0] != 'Outcome Variable'),'Due To'])]],columns=df.columns)
        df = pd.concat([df, df2])
        df['Variables'] = agg_non_mdeida(df.iloc[:,0])
        df = df[~df.Variables.isin(['Outcome Variable'])].groupby("Variables").agg({'Due To':"sum"}).reset_index()
        df=df.sort_values(['Due To'])
        
        # Append period1 outcome
        df3=pd.DataFrame([['Outcome Period 1',int(period1_outcome)]],columns=df.columns)
        df = pd.concat([df3, df])
            #Plot waterfall and save to folder
        try:
            title = "Waterfall Analysis: " + start_date_str_1 + "-" + end_date_str_1 + \
            " : " + start_date_str_2 + "-" + end_date_str_2
            save_path = './Documents_Graphs/'+model+'Dueto'+ start_date_str_1 + "-" + end_date_str_1 + \
                            start_date_str_2 + "-" + end_date_str_2+ ".png"
            matplotlib.rcParams.update({'font.size': 15})
            plot_waterfall(df.Variables.tolist(), df['Due To'].astype('float').tolist(), 
                       rotation_value= 90, net_label= "Outcome Period 2",
                       Title= title, blue_color = "#FFC220",
                       green_color = "#FFC220", red_color = '#0071CE', figsize = (15,10),
                          save_path = save_path)
            matplotlib.rcParams.update({'font.size': 13})
            return model_output_df, save_path
        except:
            msg = "Outcome Variable not found in due to file!"
            Error_msg_ls.append(msg)
            print(msg)
    except:
        msg = "Saving waterfall chart failed!"
        Error_msg_ls.append(msg)
        print(msg)
        

In [ ]:
def agg_non_mdeida(var_list):
    combined_list = []
    
    for var in var_list:
        x = var.upper()
        if ('COVID' in x) | ('EVENT' in x) | ('DAY' in x) | ('MOD' in x) | ('PRICE' in x) | ('UNEXPLAINED' in x) | ('DUMMY' in x) | ('HOLIDAY' in x) | ('SEASON' in x) | ('TREND' in x) | ('RESIDUAL' in x) | ('SARIMA' in x) | ('_SMA_DAY_' in x) :
            combined_list.append('Non Media Drivers') 
        else:
            combined_list.append(var)
    
    return combined_list
        

In [ ]:
#WRITER = pd.ExcelWriter('./WAMM_' + Brand_Name + '_Modeling_Documents.xlsx', engine = 'xlsxwriter') # location for the output file
scenario_count = 0
for model in MODEL_DICT:
    for scenario in DUE_TO_OUTCOME_DICT:
        try:
            df,image_path = get_per_scenario_due_to_outcome_dataframe(model,scenario)
            sheet_name = 'Growth Analysis'
            add_worksheet(df, sheet_name, startcol = 0, startrow = scenario_count*40+1)
            # Add waterfall chart
            worksheet = WRITER.sheets[sheet_name]
            worksheet.write_string(scenario_count*40,0,MODEL_DICT[model]['model']+': '+scenario)
            worksheet.insert_image(scenario_count*40+1,9,image_path,{'x_scale': .7, 'y_scale': .7})
            scenario_count += 1
        except:
            print("Failed to load growth analysis, check error above!")
# #WRITER.save()

### Sheet: Correlation Heatmap

In [ ]:
# Get Media Hierarchy 5 TPs
media_5 = wamm_mapping[wamm_mapping['Media Hierarchy'] == 5]['Variable Name']
media_5metrics = []
for var in media_5:
    if (var.endswith("SPEND"))|(var.endswith("RAW"))|('VIEW' in var):
        continue
    elif 'DIS' in var.upper():
        media_5metrics.append(var[:-3]+'IMP')
    elif ('SP' in var.upper())| ('SBA' in var.upper()):
        media_5metrics.append(var[:-3]+'CLK')
        
corr_var_list = list(set(media_5metrics))
# Drop variables that are not in the stack or don't have non-0 observation
corr_var_list = [var for var in corr_var_list if (var in df_stack.columns) and (sum(df_stack[var]) != 0)]

# Add outcome variables
for model in MODEL_DICT:
        corr_var_list.append(MODEL_DICT[model]['outcome_var'])
        
# Create Correlation matrix for modeling window
df_modeling_stack = df_stack[(df_stack.date <= windwow_df['Validation End'][0])&(df_stack.date >= windwow_df['Training Start'][0])]

corr = df_modeling_stack[corr_var_list].corr()
# Save Heatmap
corr.style.background_gradient(cmap='coolwarm',vmin=None, vmax=None).to_excel(WRITER,'Correlation Heatmap', engine="openpyxl", float_format= "%.3f")


### Save Excel

In [ ]:
WRITER.save()

print(round((datetime.now() - start_time).seconds/60, 2), 'mins') 
print('\n'.join(sorted(set(Error_msg_ls), key=Error_msg_ls.index)))